### 원본 dataset 이미지에 가로 왼쪽 오른쪽에 500씩 패딩하여 정사각형 만들기

In [2]:
import os
from PIL import Image
from tqdm import tqdm

def pad_images(origin_path, dest_path, padding_color='black'):
    """
    지정된 폴더 내의 이미지들에 패딩을 추가하고 새 폴더에 저장하는 함수
    
    :param origin_path: 원본 이미지가 있는 상위 폴더 경로
    :param dest_path: 패딩된 이미지를 저장할 상위 폴더 경로
    :param padding_color: 패딩에 사용할 색상 (기본값: 흰색)
    """
    total_processed = 0  # 성공적으로 처리된 이미지 수
    total_errors = 0  # 처리 중 오류가 발생한 이미지 수

    # 'training_image'와 'validation_image' 폴더를 순차적으로 처리
    for folder in ['training_image', 'validation_image']:
        src_folder = os.path.join(origin_path, folder)  # 원본 이미지 폴더 경로
        dest_folder = os.path.join(dest_path, folder)  # 패딩된 이미지를 저장할 폴더 경로
        
        # 대상 폴더가 없으면 생성
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)

        # 처리할 jpg 이미지 파일 목록 생성
        images = [f for f in os.listdir(src_folder) if f.endswith('.jpg')]
        
        # tqdm을 사용하여 진행 상황을 표시하면서 각 이미지 처리
        for filename in tqdm(images, desc=f"Processing {folder}"):
            try:
                # 이미지 열기
                with Image.open(os.path.join(src_folder, filename)) as img:
                    width, height = img.size
                    new_width = width + 1000  # 양쪽에 500픽셀씩 추가
                    
                    # 새 이미지 생성 (패딩 추가)
                    new_img = Image.new(img.mode, (new_width, height), padding_color)
                    new_img.paste(img, (500, 0))  # 원본 이미지를 새 이미지의 중앙에 붙이기
                    
                    # 패딩된 이미지 저장
                    new_img.save(os.path.join(dest_folder, filename))
                total_processed += 1
            except Exception as e:
                total_errors += 1
                print(f"\nError processing {filename}: {str(e)}")

    # 처리 결과 출력
    print(f"\nPadding completed!")
    print(f"Total images processed successfully: {total_processed}")
    print(f"Total images failed: {total_errors}")

if __name__ == "__main__":
    # 메인 경로 설정
    origin_path = "C:/Users/prohe/OneDrive/Desktop/github_manage/DCC/dataset"  # 원본 이미지가 있는 상위 폴더 경로
    dest_path = "C:/Users/prohe/OneDrive/Desktop/github_manage/DCC/b_pad_resized_data"  # 패딩된 이미지를 저장할 상위 폴더 경로

    # 이미지 패딩 함수 호출
    pad_images(origin_path, dest_path)

Processing validation_image: 100%|██████████| 951/951 [02:24<00:00,  6.57it/s]


Padding completed!
Total images processed successfully: 5021
Total images failed: 0


### 이후 224로 resize 하기

In [6]:

def resize_imgs(origin_path, dest_path):
    total_processed = 0
    total_errors = 0

    for src in ['pad_training_image', 'pad_val_image']:
        src_folder = os.path.join(origin_path, src)
        dest_folder = os.path.join(dest_path, src)
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)

        # 전체 이미지 수 계산
        total_images = sum(1 for filename in os.listdir(src_folder) if filename.endswith('.jpg'))

        print(f"\nProcessing {src} folder:")
        for filename in tqdm(os.listdir(src_folder), total=total_images, desc=f"Resizing {src}"):
            if filename.endswith('.jpg'):
                try:
                    input_path = os.path.join(src_folder, filename)
                    output_path = os.path.join(dest_folder, filename)
                    img = Image.open(input_path)
                    img_resized = img.resize((224, 224), Image.LANCZOS)
                    img_resized.save(output_path)
                    total_processed += 1
                except Exception as e:
                    total_errors += 1
                    print(f"\nError processing {filename}: {str(e)}")

    print(f"\nResizing completed!")
    print(f"Total images processed successfully: {total_processed}")
    print(f"Total images failed: {total_errors}")


if __name__ == "__main__":
    # path 지정
    MAIN_PATH = "/home/shinds/my document"
    origin_path = os.path.join(MAIN_PATH, "pad")
    dest_folder = os.path.join(MAIN_PATH, "pad_resized_data")

    # 새로 resized 된 파일을 저장할 폴더가 없다면 생성
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    resize_imgs(origin_path, dest_folder)


Processing pad_training_image folder:


Resizing pad_training_image: 100%|██████████| 4070/4070 [06:04<00:00, 11.16it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/home/shinds/my document/pad/pad_validation_image'

### 이미지 카테고리화

In [9]:
import shutil
def categorize_imgs(src, dest_dir):
    # src_folder 지정
    src_folder = os.path.join('./pad_resized_data', src)
    
    # dest_folder 지정(폴더가 없다면 생성)
    dest_folder = os.path.join(dest_dir, src)
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    
    # 오류 발생을 대비하여 처리된 이미지 수 추적
    processed_count = 0
    error_count = 0
    
    # src_folder의 모든 파일을 가져옴
    for filename in os.listdir(src_folder):
        if filename.endswith('.jpg'):
            try:
                # 파일명에서 style 정보 추출
                style = filename.split('_')[-2]

                # style에 해당하는 폴더가 없다면 생성
                style_folder = os.path.join(dest_folder, style)
                if not os.path.exists(style_folder):
                    os.makedirs(style_folder)
                
                # 원본 이미지 복사(copy2: metadata까지 복사)
                src_path = os.path.join(src_folder, filename)
                dest_path = os.path.join(style_folder, filename)
                shutil.copy2(src_path, dest_path)

                # 정상적으로 처리하였을 경우 count 증가
                processed_count += 1
    
            except Exception as e:
                # 에러가 발생하여 변환에 실패하면 에러 메시지 출력
                error_count += 1
                print(f"Error processing {filename}: {str(e)}")

    print(f"Images are categorized successfully! ==> Total images processed: {processed_count} / Total images failed: {error_count}")


if __name__ == "__main__":
    # 데이터가 묶여서 저장될 폴더
    dest_dir = './categorized_data'
    # 새로 만들어질 폴더가 없다면 생성
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # source data가 있는 폴더 지정
    train_path = "pad_training_image"
    val_path = "pad_val_image"

    # style 별로 이미지 분류
    categorize_imgs(train_path, dest_dir)
    categorize_imgs(val_path, dest_dir)

Images are categorized successfully! ==> Total images processed: 4070 / Total images failed: 0
Images are categorized successfully! ==> Total images processed: 951 / Total images failed: 0
